In [4]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ChargeDataSet import *
from MS_L_IDs import *
from ShowDF import *
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from AllMS2Data import *
from feat_ms2_Gauss import *

In [26]:
#from ms2_SamplesAligment import *
ResultsFolder = home+'/ms2_spectra_Summ-20260107'
ms2Folder = home+'/ms2_spectra-20260107'
AlignedSamplesDF = ms2_SamplesAligment(ResultsFolder = ResultsFolder,
                                       mz_min = 254,
                                       mz_max = 269,
                                       RT_min = 0,
                                       RT_max = 2000,
                                       RT_tol = 30,
                                       mz_Tol = 2e-3,
                                       min_Int_Frac = 1,
                                       cos_tol = 0.9,
                                       min_N_ms2_spectra = 3,
                                       ToReplace = 'mzML-ms2Summary.xlsx',
                                       ms2Folder = ms2Folder,
                                       ToAdd = 'mzML',
                                       saveAlignedTable = False,
                                       name = "SamplesAligment",
                                       Norm2One = True)

In [27]:
ShowDF(AlignedSamplesDF)

,mz_(Da),mz_std_(Da),N_ms2-spectra,mean CosSim,std CosSim,Normal test,mz_ConfidenceInterval_(Da),mz_ConfidenceInterval_(ppm),RT_(s),min_RT_(s),max_RT_(s),feat_id,11011,11012,11013,11111,11121,11131,11211,11212,11213,11221,11222,11223,11231,11232,11233,12011,12012,12013,12111,12112,12113,12211,12212,12213,12221,12222,12223,12231,12232,12233,12311,12312,12313,13011,13012,13013,13111,13112,13113,13211,13212,13213,13221,13222,13223,13231,13232,13233,13311,13312,13313,21001,21002,21003
0,254.059,0.000101777,73,0.979648,0.0308081,0,3.15178e-05,0.124057,139.452,133.379,152.157,2478,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
1,254.09,9.2255e-05,14,1,0,0,7.42713e-05,0.292303,118.543,117.062,120.65,2583,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0
2,254.175,0.000105232,54,0.961387,0.0552784,0,3.82611e-05,0.150531,121.586,119.41,130.003,4,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,0
3,255.034,6.638e-05,10,0.982982,0.0109365,0,6.82179e-05,0.267486,105.592,103.572,106.941,9,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,255.084,0.000132074,7,0.989997,0.011864,0,0.000185072,0.725535,153.894,151.286,157.228,2697,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,255.116,0.000106891,36,0.988196,0.0118753,0,4.85251e-05,0.190208,82.341,78.6864,87.5439,2491,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
6,255.12,0.00012494,17,0.901922,0.121846,0,8.8507e-05,0.346923,180.091,178.076,183.994,12,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,1,0,0,0
7,256.144,9.35042e-05,7,0.996721,0.0022269,0,0.000131025,0.511529,201.286,200.698,202.078,2626,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,256.263,0.000103951,23,1,0,0,6.10973e-05,0.238416,81.5039,11.0281,179.523,19,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,257.092,9.81966e-05,6,0.965658,0.0340709,0,0.000161643,0.628736,220.474,214.842,225.547,2721,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
from AdjacencyList_from_matrix import *
AdjacencyList_Features,features_ids = AdjacencyList_from_matrix(CosineMat = CosineMat2.copy(),
                                                                N_ms2_spectra = len(CosineMat2),
                                                                cos_tol = 0.9)

In [42]:
from CommunityBlocks import *
Feature_Modules = CommunityBlocks(AdjacencyList_Features = AdjacencyList_Features)

In [211]:
ShowDF(CosineMat2[np.ix_(Feature_Modules[0],Feature_Modules[0])])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51
0,0,0.999382,0.999489,0.930211,0.971054,0.992125,0.988309,0.978759,0.991578,0.956827,0.960127,0.953627,0.959445,0.981805,0.99327,0.93759,0.99271,0.970875,0.981261,0.993937,0.992659,0.947808,0.992803,0.991061,0.994993,0.98744,0.99322,0.994116,0.931594,0.961763,0.961172,0.975195,0.988535,0.979435,0.989202,0.980023,0.975448,0.977438,0.975364,0.962641,0.975788,0.978414,0.9818,0.984287,0.960498,0.993082,0.988153,0.99367,0.996456,0.993915,0.993595,0.990798
1,0.999382,0,0.998993,0.92928,0.966027,0.989667,0.987757,0.978091,0.990252,0.956075,0.958423,0.952017,0.956122,0.9785,0.991287,0.936331,0.990991,0.970231,0.978413,0.990996,0.988846,0.946636,0.991045,0.989883,0.991785,0.987052,0.991264,0.990912,0.929967,0.964284,0.959959,0.97164,0.983643,0.978289,0.984443,0.977675,0.973688,0.97416,0.974133,0.963002,0.972405,0.975714,0.979274,0.983572,0.960525,0.989453,0.98456,0.991579,0.993166,0.991383,0.991406,0.98969
2,0.999489,0.998993,0,0.925862,0.970075,0.988728,0.983317,0.974085,0.987224,0.952179,0.956267,0.949724,0.95696,0.979182,0.989463,0.933478,0.988683,0.966058,0.979295,0.990938,0.990406,0.943565,0.988808,0.986586,0.993354,0.982315,0.989391,0.991509,0.927827,0.954316,0.959777,0.973039,0.987218,0.974997,0.987768,0.976599,0.971785,0.975013,0.971021,0.960066,0.973263,0.975296,0.978516,0.979698,0.95851,0.99067,0.985737,0.990168,0.995146,0.99057,0.989962,0.986264
3,0.930211,0.92928,0.925862,0,0.966942,0.935263,0.939078,0.978447,0.939167,0.995639,0.995055,0.997121,0.989737,0.974774,0.938211,0.999706,0.938709,0.987201,0.977718,0.935104,0.930478,0.998531,0.938646,0.939251,0.929913,0.938891,0.938268,0.933552,0.999429,0.925802,0.8979,0.979373,0.922335,0.980558,0.923492,0.98112,0.9854,0.979517,0.985418,0.904748,0.980458,0.981739,0.978604,0.967615,0.900068,0.931602,0.961324,0.937292,0.930554,0.936679,0.937712,0.939275
4,0.971054,0.966027,0.970075,0.966942,0,0.972659,0.958501,0.975308,0.965911,0.975542,0.983953,0.981347,0.993064,0.99529,0.971155,0.972668,0.969164,0.976714,0.988792,0.977058,0.980613,0.976092,0.969468,0.964603,0.971831,0.956761,0.97096,0.97771,0.972541,0.915969,0.917097,0.995792,0.982549,0.980941,0.982459,0.990597,0.985493,0.994886,0.982288,0.908854,0.996119,0.992846,0.991424,0.972199,0.906611,0.980007,0.993243,0.970906,0.97638,0.974795,0.972606,0.963962
5,0.992125,0.989667,0.988728,0.935263,0.972659,0,0.993982,0.98349,0.99673,0.962167,0.964821,0.958347,0.963001,0.985511,0.997972,0.942458,0.997594,0.976377,0.979646,0.997973,0.996076,0.952799,0.99766,0.996313,0.992504,0.993223,0.997941,0.99714,0.936167,0.969425,0.950624,0.977613,0.991172,0.984638,0.991936,0.984388,0.979063,0.980172,0.980483,0.952351,0.979403,0.982523,0.986052,0.988653,0.948316,0.996631,0.991697,0.997249,0.994594,0.998237,0.998155,0.996097
6,0.988309,0.987757,0.983317,0.939078,0.958501,0.993982,0,0.989016,0.999302,0.967014,0.965425,0.959315,0.956335,0.979117,0.99761,0.944865,0.998412,0.981833,0.976256,0.993319,0.987496,0.955686,0.998304,0.999543,0.987783,0.999969,0.997699,0.991424,0.936872,0.989002,0.956083,0.970198,0.977715,0.987909,0.979085,0.982274,0.979709,0.974003,0.983337,0.965025,0.97261,0.978816,0.983156,0.994065,0.959767,0.988884,0.984071,0.996557,0.988044,0.995415,0.996867,0.99964
7,0.978759,0.978091,0.974085,0.978447,0.975308,0.98349,0.989016,0,0.988498,0.993225,0.991863,0.989123,0.983932,0.991155,0.986971,0.981699,0.987705,0.998486,0.99208,0.982941,0.977378,0.987643,0.987607,0.988703,0.977468,0.988949,0.987053,0.982086,0.976771,0.977456,0.949285,0.989011,0.967945,0.999203,0.96927,0.995876,0.997594,0.991254,0.998521,0.958186,0.989673,0.99402,0.995307,0.998855,0.952844,0.978708,0.988377,0.987171,0.979122,0.984923,0.986283,0.988784
8,0.991578,0.990252,0.987224,0.939167,0.965911,0.99673,0.999302,0.988498,0,0.966734,0.966862,0.960594,0.960704,0.983415,0.999495,0.945535,0.999819,0.981332,0.979331,0.996936,0.992695,0.9561

In [16]:
IntramoduleSimilarity,CompactCosineTen,Modules,ConflictiveNeighborsList = OverlappingClustering(Feature_Modules = Feature_Modules,
                                                                                                CosineMat = CosineMat2.copy(),
                                                                                                percentile = 10,
                                                                                                cos_tol = 0.9)

In [17]:
ConflictiveNeighborsList

[[], [], [], []]

In [18]:
Modules

[[13],
 [1, 10],
 [27, 60, 53],
 [56,
  34,
  33,
  4,
  5,
  36,
  41,
  43,
  12,
  45,
  14,
  15,
  16,
  19,
  21,
  25,
  59,
  0,
  2,
  3,
  7,
  8,
  9,
  11,
  17,
  18,
  20,
  23,
  24,
  26,
  28,
  29,
  30,
  31,
  32,
  35,
  37,
  38,
  39,
  44,
  47,
  48,
  49,
  50,
  51,
  52,
  54,
  55,
  57,
  58,
  6,
  40,
  42,
  46,
  22]]

In [50]:
New_Modules

[[0], [1], [3], [2, 4, 6], [5]]

In [80]:
Feature_Modules

[[13],
 [1, 10],
 [56, 34],
 [27, 60, 53],
 [33, 4, 5, 36, 41, 43, 12, 45, 14, 15, 16, 19, 21, 25, 59],
 [6, 40, 42, 46, 22],
 [0,
  2,
  3,
  7,
  8,
  9,
  11,
  17,
  18,
  20,
  23,
  24,
  26,
  28,
  29,
  30,
  31,
  32,
  35,
  37,
  38,
  39,
  44,
  47,
  48,
  49,
  50,
  51,
  52,
  54,
  55,
  57,
  58]]

In [ ]:
NeighborhoodSizeVec = np.array([len(x) for x in AdjacencyList_Features])

In [37]:
print(NewAdjacencyList)

[[0], [1], [4, 6, 2], [3], [5, 6, 4], [4, 6, 5], [4, 5, 6]]


In [39]:
odules

[[0], [1], [3], [2, 4, 6], [5]]

In [19]:
ShowDF(IntramoduleSimilarity)
ShowDF(CompactCosineTen[:,:,0])
ShowDF(CompactCosineTen[:,:,1])
ShowDF(CompactCosineTen[:,:,2])

,0,1,2,3,4
0,1,1,1,1,0
1,2,0.953068,0.953068,0.953068,1
2,3,0.998146,0.998146,1,2
3,55,0.909034,0.98042,0.997546,3


,0,1,2,3
0,0,0,0,0
1,0,0,0.592472,0.329263
2,0,0.592472,0,0.537811
3,0,0.329263,0.537811,0


,0,1,2,3
0,0,0,0,0
1,0,0,0.700244,0.399152
2,0,0.700244,0,0.616242
3,0,0.399152,0.616242,0


,0,1,2,3
0,0,0,0,0
1,0,0,0.808966,0.527648
2,0,0.808966,0,0.679836
3,0,0.527648,0.679836,0


In [27]:
def ClustersCentroid(CosineMat,
                     Cluster):
    N_membersCluster = len(Cluster[0])        
    Centroid = sum(CosineMat[Cluster[0],:])/N_membersCluster
    Centroid[Cluster[0]] = 0
    return Centroid

In [28]:
import numpy as np
def EvaluateSimilarities(CosineMat,
                         Cluster,
                         ZeroRow,
                         Centroid,
                         CosTolDif = 0.15):    
    ClusterMembers = Cluster[0] 
    ClusterCoherence = Cluster[1]
    ClusterExternalAffinity = Cluster[2]
    NewMemberCosineVec = CosineMat[ZeroRow,:]    
    NodesSet = set(np.arange(len(NewMemberCosineVec),dtype='int').tolist())
    NewClusterCoherence = float(min(NewMemberCosineVec[ClusterMembers].tolist()))
    NewClusterCoherence = min([NewClusterCoherence,ClusterCoherence])
    NoNeighborsSet = NodesSet - set(ClusterMembers)
    ClusterExternalAffinity = np.max(CosineMat[np.ix_(ClusterMembers,list(NoNeighborsSet))])
    r,c = np.where(CosineMat[np.ix_(ClusterMembers,list(NoNeighborsSet))] == ClusterExternalAffinity)
    print(ClusterMembers[r[0]],list(NoNeighborsSet)[c[0]])
    NewClusterExternalAffinity = float(max(NewMemberCosineVec[list(NoNeighborsSet)]))
    NewClusterExternalAffinity = max([NewClusterExternalAffinity,ClusterExternalAffinity])
    Contrast = NewClusterCoherence-NewClusterExternalAffinity+CosTolDif
    merge = Contrast>=0
    if merge:
        Cluster[0] = ClusterMembers + [ZeroRow]
        Cluster[1] = NewClusterCoherence
        Cluster[2] = NewClusterExternalAffinity
    else:
        Centroid[ZeroRow] = 0
    return [merge,Cluster,Centroid]
    

In [29]:
import numpy as np
#from EvaluateSimilarities import *
#from UpdateCosineMat import *
def BinaryClustering(ClusterRow,
                     CosineMat,
                     Cluster,
                     Centroid,
                     iterations_since_addition,
                     CosTolDif = 0.15):
    maxSim = np.max(Centroid)
    maxSimLoc = np.where(Centroid == maxSim)[0][0]
    ZeroRow = int(maxSimLoc)
    print(ZeroRow)
    merge,Cluster,Centroid = EvaluateSimilarities(CosineMat = CosineMat,
                                                  Cluster = Cluster,
                                                  Centroid = Centroid,
                                                  ZeroRow = ZeroRow,
                                                  CosTolDif = CosTolDif)
    print(merge)
    if not merge:
        print(Cluster[1],Cluster[2])
        iterations_since_addition += 1
        return [CosineMat,Cluster,Centroid,iterations_since_addition]
    Centroid = ClustersCentroid(CosineMat = CosineMat,
                                Cluster = Cluster)  
    return [CosineMat,Cluster,Centroid,iterations_since_addition]

In [32]:
ClusterRow

3

In [31]:
from MostPopularSpectrum import *
CosineMat = CompactCosineTen[:,:,1].copy()
ClusterRow = MostPopularSpectrum(CosineMat = CosineMat)
CosTolDif = 0
Cluster = [[ClusterRow],1,0]
Centroid = CosineMat[ClusterRow,:].copy()
N_spectra = len(Centroid)
N_NoMembersCluster = N_spectra-1
iterations_since_addition = 0
while iterations_since_addition <= N_NoMembersCluster:
    CosineMat,Cluster,Centroid,iterations_since_addition = BinaryClustering(ClusterRow = ClusterRow,
                                                  CosineMat = CosineMat,
                                                  Cluster = Cluster,
                                                  Centroid = Centroid,
                                                  CosTolDif = CosTolDif,
                                                  iterations_since_addition = iterations_since_addition)
    N_membersCluster = len(Cluster[0])        
    N_NoMembersCluster = N_spectra-N_membersCluster

2
3 2
True
0
3 0
False
0.9920708779797164 0.9920708779797164
1
3 0
False
0.9920708779797164 0.9920708779797164
5
3 0
False
0.9920708779797164 0.9920708779797164
4
3 0
False
0.9920708779797164 0.9920708779797164
0
3 0
False
0.9920708779797164 0.9920708779797164


In [12]:
len(Cluster[0])

70

In [23]:
ClusterMembers = Cluster[0]
NodesSet = set(np.arange(len(CosineMat[:,0]),dtype='int').tolist())
NoNeighborsSet = NodesSet - set(ClusterMembers)

In [24]:
len(NodesSet)

75

In [46]:
NoNeighborsSet

{4, 10, 23, 56, 61}

In [26]:
Cluster[1],Cluster[2]

(0.8439780495053187, 0.9854002833373364)

In [10]:

MaxSimAdjacencyList = [[[j],0,0] for j in range(CosineMat.shape[0])]

for c in np.arange(10):
    maxSim = np.max(CosineMat)
    maxSimLoc = np.where(CosineMat == maxSim)[0]
    print(maxSim, maxSimLoc)
    ZeroRow = int(maxSimLoc[0])
    ClusterRow = int(maxSimLoc[1])
    CosineMat = UpdateCosineMat(CosineMat = CosineMat,
                                ZeroRow = ZeroRow,
                                ClusterRow = ClusterRow,
                                MaxSimAdjacencyList = MaxSimAdjacencyList)    
    MaxSimAdjacencyList = FillMaxSimAdjacencyList(MaxSimAdjacencyList = MaxSimAdjacencyList,
                                                  ZeroRow = ZeroRow,
                                                  ClusterRow = ClusterRow,
                                                  CosineMat = CosineMat)

# add intra and inter cluster similarity

1.0 [27 60]
[27, 60]
sim 0.5927680921377605
0.9999991580078778 [18 31]
[18, 31]
sim 0.9095121332768155
0.9999982245132851 [20 26]
[26, 20]
sim 0.8920650795708374
0.9999942519361951 [28 58]
[58, 28]
sim 0.8733109316681198
0.999978042838328 [37 39]
[37, 39]
sim 0.8555713986776237
0.9999687621245179 [ 8 30]
[8, 30]
sim 0.8368554555094929
0.9999597166216421 [11 58]
[58, 11, 28]
sim 0.8371396275797762
0.9999589868344891 [24 50]
[24, 50]
sim 0.8083574277335184
0.9999428086668847 [31 57]
[57, 18, 31]
sim 0.8049282446463337
0.9999106985170966 [40 47]
[40, 47]
sim 0.7810415095860594


In [ ]:

MaxSimAdjacencyList = [[[j],0,0] for j in range(CosineMat.shape[0])]

for c in np.arange(10):
    maxSim = np.max(CosineMat)
    maxSimLoc = np.where(CosineMat == maxSim)[0]
    print(maxSim, maxSimLoc)
    ZeroRow = int(maxSimLoc[0])
    ClusterRow = int(maxSimLoc[1])
    CosineMat = UpdateCosineMat(CosineMat = CosineMat,
                                ZeroRow = ZeroRow,
                                ClusterRow = ClusterRow,
                                MaxSimAdjacencyList = MaxSimAdjacencyList)    
    MaxSimAdjacencyList = FillMaxSimAdjacencyList(MaxSimAdjacencyList = MaxSimAdjacencyList,
                                                  ZeroRow = ZeroRow,
                                                  ClusterRow = ClusterRow,
                                                  CosineMat = CosineMat)

# add intra and inter cluster similarity

In [11]:
MaxSimAdjacencyList

[[[0], 0, 0],
 [[1], 0, 0],
 [[2], 0, 0],
 [[3], 0, 0],
 [[4], 0, 0],
 [[5], 0, 0],
 [[6], 0, 0],
 [[7], 0, 0],
 [[8, 30], 0, 0],
 [[9], 0, 0],
 [[10], 0, 0],
 [[58, 11, 28], 0, 0],
 [[12], 0, 0],
 [[13], 0, 0],
 [[14], 0, 0],
 [[15], 0, 0],
 [[16], 0, 0],
 [[17], 0, 0],
 [[57, 18, 31], 0, 0],
 [[19], 0, 0],
 [[26, 20], 0, 0],
 [[21], 0, 0],
 [[22], 0, 0],
 [[23], 0, 0],
 [[24, 50], 0, 0],
 [[25], 0, 0],
 [[26, 20], 0.8920650795708374, 0],
 [[27, 60], 0, 0],
 [[58, 11, 28], 0, 0],
 [[29], 0, 0],
 [[8, 30], 0.8368554555094929, 0],
 [[57, 18, 31], 0.9095121332768155, 0],
 [[32], 0, 0],
 [[33], 0, 0],
 [[34], 0, 0],
 [[35], 0, 0],
 [[36], 0, 0],
 [[37, 39], 0, 0],
 [[38], 0, 0],
 [[37, 39], 0.8555713986776237, 0],
 [[40, 47], 0, 0],
 [[41], 0, 0],
 [[42], 0, 0],
 [[43], 0, 0],
 [[44], 0, 0],
 [[45], 0, 0],
 [[46], 0, 0],
 [[40, 47], 0.7810415095860594, 0],
 [[48], 0, 0],
 [[49], 0, 0],
 [[24, 50], 0.8083574277335184, 0],
 [[51], 0, 0],
 [[52], 0, 0],
 [[53], 0, 0],
 [[54], 0, 0],
 [[55], 

In [212]:
L[27]=[]

In [214]:
    ClusteringMatrix[ZeroRow,ClusterRow] = maxSim
    ClusteringMatrix[ClusterRow,ZeroRow] = maxSim

list

In [189]:
ShowDF(ClusteringMatrix)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0,0,0,0.999489,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0.999187,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.999489,0,0.999187,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.999706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.999969,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.998855,0,0,0,0,0,0,0,0,0,0,0,0


In [158]:
Centroid[maxSimLoc]

array([0.5, 0.5])

In [161]:

Centroid

array([0.63934087, 0.59302157, 0.6218247 , 0.65064119, 0.55689761,
       0.54045417, 0.71425398, 0.63099839, 0.55790254, 0.55560633,
       0.80896569, 0.58851087, 0.54825551, 0.        , 0.58550275,
       0.57848898, 0.64544121, 0.65693282, 0.61391167, 0.55667658,
       0.60377173, 0.55184427, 0.628119  , 0.64995032, 0.68175453,
       0.55897606, 0.60527283, 0.5       , 0.58274031, 0.64716233,
       0.55132545, 0.6128868 , 0.65836162, 0.56712481, 0.4290214 ,
       0.53604922, 0.66067294, 0.71969663, 0.57524692, 0.71507995,
       0.62030986, 0.59387326, 0.65177412, 0.57619848, 0.47652551,
       0.65653682, 0.6325408 , 0.6278915 , 0.55972489, 0.48432588,
       0.67510083, 0.67106376, 0.61624234, 0.99814621, 0.66380288,
       0.63472564, 0.        , 0.62179534, 0.57998157, 0.45407323,
       0.        ])

(61, 61)

In [153]:
CosineMat2 = CosineMat.copy()
CosineMat2[ZeroRow,:] = 0
CosineMat2[:,ZeroRow] = 0


In [256]:
ShowDF(CosineMat)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [143]:
arr_rows_deleted = np.delete(CosineMat, maxSimLoc, axis=0)
arr_rows_deleted = np.delete(arr_rows_deleted, maxSimLoc, axis=1)

In [144]:
ShowDF(arr_rows_deleted)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,0,0.379143,0.999382,0.999489,0.814994,0.930211,0.971054,0.992125,0.988309,0.978759,0.517205,0.991578,0.956827,0,0.960127,0.953627,0.959445,0.981805,0.99327,0.93759,0.99271,0.970875,0.981261,0.993937,0.992659,0.947808,0.992803,0.991061,0.994993,0.98744,0.99322,0.994116,0.931594,0.961763,0.961172,0.975195,0.988535,0.979435,0.989202,0.980023,0.975448,0.977438,0.975364,0.962641,0.975788,0.978414,0.9818,0.984287,0.960498,0.993082,0.988153,0.99367,0.662266,0.996456,0.993915,0.761072,0.993595,0.990798,0.804015
1,0.379143,0,0.368755,0.385844,0.330252,0.320501,0.423568,0.374196,0.330848,0.329487,0.953068,0.349,0.325127,0,0.347216,0.343056,0.382761,0.389575,0.364063,0.330121,0.35805,0.327256,0.372488,0.385435,0.404295,0.331485,0.35894,0.345578,0.383781,0.326948,0.363455,0.390423,0.336317,0.254419,0.317889,0.391793,0.426796,0.341134,0.424058,0.367857,0.35218,0.386516,0.341698,0.28259,0.38934,0.37511,0.372353,0.331929,0.287216,0.400349,0.397955,0.365445,0.591922,0.393649,0.376406,0,0.368738,0.343942,0.269275
2,0.999382,0.368755,0,0.998993,0.811939,0.92928,0.966027,0.989667,0.987757,0.978091,0.503035,0.990252,0.956075,0,0.958423,0.952017,0.956122,0.9785,0.991287,0.936331,0.990991,0.970231,0.978413,0.990996,0.988846,0.946636,0.991045,0.989883,0.991785,0.987052,0.991264,0.990912,0.929967,0.964284,0.959959,0.97164,0.983643,0.978289,0.984443,0.977675,0.973688,0.97416,0.974133,0.963002,0.972405,0.975714,0.979274,0.983572,0.960525,0.989453,0.98456,0.991579,0.645067,0.993166,0.991383,0.772183,0.991406,0.98969,0.803561
3,0.999489,0.385844,0.998993,0,0.81308,0.925862,0.970075,0.988728,0.983317,0.974085,0.526346,0.987224,0.952179,0,0.956267,0.949724,0.95696,0.979182,0.989463,0.933478,0.988683,0.966058,0.979295,0.990938,0.990406,0.943565,0.988808,0.986586,0.993354,0.982315,0.989391,0.991509,0.927827,0.954316,0.959777,0.973039,0.987218,0.974997,0.987768,0.976599,0.971785,0.975013,0.971021,0.960066,0.973263,0.975296,0.978516,0.979698,0.95851,0.99067,0.985737,0.990168,0.673458,0.995146,0.99057,0.747461,0.989962,0.986264,0.799959
4,0.814994,0.330252,0.811939,0.81308,0,0.960219,0.908762,0.817853,0.81115,0.885371,0.450511,0.815281,0.931083,0,0.93556,0.943191,0.940795,0.898619,0.8179,0.956826,0.816946,0.905371,0.902634,0.820242,0.82083,0.946316,0.817095,0.814581,0.815797,0.810131,0.817809,0.82014,0.964041,0.783632,0.775928,0.913955,0.819472,0.891977,0.819768,0.902526,0.908791,0.910103,0.903895,0.773407,0.913334,0.907737,0.898467,0.863688,0.770768,0.82083,0.873772,0.817488,0.573847,0.818508,0.819459,0.603027,0.818555,0.814234,0.992071
5,0.930211,0.320501,0.92928,0.925862,0.960219,0,0.966942,0.935263,0.939078,0.978447,0.437209,0.939167,0.995639,0,0.995055,0.997121,0.989737,0.974774,0.938211,0.999706,0.938709,0.987201,0.977718,0.935104,0.930478,0.998531,0.938646,0.939251,0.929913,0.938891,0.938268,0.933552,0.999429,0.925802,0.8979,0.979373,0.922335,0.980558,0.923492,0.98112,0.9854,0.979517,0.985418,0.904748,0.980458,0.981739,0.978604,0.967615,0.900068,0.931602,0.961324,0.937292,0.56236,0.930554,0.936679,0.768228,0.937712,0.939275,0.963494
6,0.971054,0.423568,0.966027,0.970075,0.908762,0.966942,0,0.972659,0.958501,0.975308,0.577807,0.965911,0.975542,0,0.983953,0.981347,0.993064,0.99529,0.971155,0.972668,0.969164,0.976714,0.988792,0.977058,0.980613,0.976092,0.969468,0.964603,0.971831,0.956761,0.97096,0.97771,0.972541,0.915969,0.917097,0.995792,0.982549,0.980941,0.982459,0.990597,0.985493,0.994886,0.982288,0.908854,0.996119,0.992846,0.991424,0.972199,0.906611,0.980007,0.993243,0.970906,0.733052,0.97638,0.974795,0.674795,0.972606,0.963962,0.885769
7,0.992125,0.374196,0.989667,0.988728,0.817853,0.935263,0.972659,0,0.993982,0.98349,0.510456,0.99673,0.962167,0,0.964821,0.958347,0.963001,0.985511,0.997972,0.942458,0.997594,0.976377,0.979646,0.997973,0.996076,0.952799,0.99766,0.996313,0.992504,0.993223

In [123]:
ShowDF(AlignedSamplesDF)

,mz_(Da),mz_std_(Da),N_ms2-spectra,mean CosSim,std CosSim,Normal test,mz_ConfidenceInterval_(Da),mz_ConfidenceInterval_(ppm),RT_(s),min_RT_(s),max_RT_(s),feat_id,11011,11012,11013,11111,11121,11131,11211,11212,11213,11221,11222,11223,11231,11232,11233,12011,12012,12013,12111,12112,12113,12211,12212,12213,12221,12222,12223,12231,12232,12233,12311,12312,12313,13011,13012,13013,13111,13112,13113,13211,13212,13213,13221,13222,13223,13231,13232,13233,13311,13312,13313,21001,21002,21003
0,267.134,0.000103918,8,0.866492,0.0849938,0,0.000128573,0.481304,118.63,117.794,119.92,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0
1,267.156,8.8321e-05,7,1,0,0,0.000123762,0.463257,243.194,241.638,245.646,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,267.17,0.000121914,36,0.80345,0.252522,0,5.5345e-05,0.207153,117.246,115.327,121.06,14,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,1,0,1,1,1,0,0,0,1,1,1,1,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0
3,267.17,6.767e-05,6,0.801137,0.144458,0,0.000111393,0.416936,91.9068,89.9393,94.6819,13,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,267.17,8.92474e-05,60,0.901235,0.0971595,0,3.06682e-05,0.114789,89.4011,83.6555,98.1945,15,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0


In [7]:
ShowDF(AlignedSamplesDF)

,mz_(Da),mz_std_(Da),N_ms2-spectra,mean CosSim,std CosSim,Normal test,mz_ConfidenceInterval_(Da),mz_ConfidenceInterval_(ppm),RT_(s),min_RT_(s),max_RT_(s),feat_id,11011,11012,11013,11111,11121,11131,11211,11212,11213,11221,11222,11223,11231,11232,11233,12011,12012,12013,12111,12112,12113,12211,12212,12213,12221,12222,12223,12231,12232,12233,12311,12312,12313,13011,13012,13013,13111,13112,13113,13211,13212,13213,13221,13222,13223,13231,13232,13233,13311,13312,13313,21001,21002,21003
0,267.134,0.000103918,8,0.866492,0.0849938,0,0.000128573,0.481304,118.63,117.794,119.92,46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0
1,267.156,8.8321e-05,7,1,0,0,0.000123762,0.463257,243.194,241.638,245.646,43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,267.17,0.000127595,28,0.957416,0.0471568,0,6.68102e-05,0.250066,116.847,115.327,118.166,17,0,0,0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0
3,267.17,9.27392e-05,53,0.945821,0.0314307,0,3.40599e-05,0.127484,88.9918,83.6555,97.0464,18,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0
4,267.17,5.55512e-05,7,1,0,0,7.78426e-05,0.29136,92.5002,88.4232,98.1945,20,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
5,267.17,9.36844e-05,7,0.912775,0.0538346,0,0.000131278,0.491364,118.294,117.22,119.513,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,268.104,9.87521e-05,72,0.999302,0.00226443,0,3.08043e-05,0.114897,67.2595,60.3324,79.391,29,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0
7,268.104,7.96116e-05,8,0.967245,0.0344987,0,9.84997e-05,0.367394,93.1278,88.8733,98.0032,28,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,268.154,0.000162389,11,0.955066,0.0307792,0,0.000155174,0.578674,112.131,111.043,112.947,34,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,268.154,9.36662e-05,7,1,0,0,0.000131252,0.489466,114.123,113.46,114.987,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0


In [26]:
DataFolder='/home/edwin/Documents/16.02/data'
ResultsFolder=home+'/Results-20260110'
ms2Folder='ms2_spectra-20260107'
SFindicator='-ms2Summary.xlsx'
Batch_ms2Redundancy_and_MS1Refine(DataFolder = DataFolder,
                                  ResultsFolder = ResultsFolder,
                                  ms2FolderName = ms2Folder,
                                  SFindicator='-ms2Summary.xlsx',
                                  ToReplace='mzML-ms2Summary.xlsx',
                                  min_RT=0,
                                  max_RT=1500,
                                  min_mz=0,
                                  max_mz=1200,
                                  mz_col=1,
                                  RT_col=2,
                                  RT_tol=20,
                                  mz_Tol=1e-2,
                                  sample_id_col=-1,
                                  ms2_spec_id_col=0,
                                  ToAdd='mzML',
                                  min_Int_Frac=2,
                                  cos_tol=0.9,
                                  mz_std=2e-3,
                                  MS2_to_MS1_ratio=10,
                                  stdDistance=3,
                                  MaxCount=3,
                                  Points_for_regression=5,
                                  minSignals=7,
                                  overwrite=False,
                                  LogFileName='LogFile_ms2Refine.csv')

---
21111_noSPE.mzML


/tmp/ipykernel_13184/1513245385.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mod_loc=int(np.where(mostInt_ms2Frag_Filter)[0])


Saved: /home/edwin/0-GitHubProjects/Codding/ms2Gauss/Results-20260110/21111_noSPE.mzML.xlsx
0 0
---
12313.mzML


/tmp/ipykernel_13184/1513245385.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mod_loc=int(np.where(mostInt_ms2Frag_Filter)[0])


Saved: /home/edwin/0-GitHubProjects/Codding/ms2Gauss/Results-20260110/12313.mzML.xlsx
1 1
---
12212.mzML


...

Saved: /home/edwin/0-GitHubProjects/Codding/ms2Gauss/Results-20260110/11013.mzML.xlsx
63 95
---
11222.mzML


/tmp/ipykernel_13184/1513245385.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mod_loc=int(np.where(mostInt_ms2Frag_Filter)[0])


error 595
Saved: /home/edwin/0-GitHubProjects/Codding/ms2Gauss/Results-20260110/11222.mzML.xlsx
64 96
---
13313.mzML


/tmp/ipykernel_13184/1513245385.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mod_loc=int(np.where(mostInt_ms2Frag_Filter)[0])


error 419
Saved: /home/edwin/0-GitHubProjects/Codding/ms2Gauss/Results-20260110/13313.mzML.xlsx
65 98


In [8]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ChargeDataSet_in_AnotherFolder import *
#from Retrieve_csv_ms2Spectra import *
from ms2_SpectralRedundancy import *
from feat_ms2_Gauss import *
DataFolder='/home/edwin/Documents/16.02/data'
ResultsFolder=home+'/Results-20260109'
ms2Folder=home+'/ms2_spectra-20260107'
SFindicator='-ms2Summary.xlsx'
for datasetname in os.listdir(DataFolder):
    while True:
        try:
            print(datasetname)
            SummaryFile=datasetname.replace('.','')+SFindicator
            DataSet=ChargeDataSet_in_AnotherFolder(datasetname=datasetname,
                                                   DataFolder=DataFolder)
            MS1IDVec=MS_L_IDs(DataSet=DataSet,
                              Level=1)
            SummMS2=ms2_SpectralRedundancy(SummaryFile=SummaryFile,
                                           min_RT=0,
                                           max_RT=1500,
                                           min_mz=0,
                                           max_mz=1200,
                                           ms2FolderName='ms2_spectra-20260107',
                                           ToReplace='mzML-ms2Summary.xlsx',
                                           mz_col=1,
                                           RT_col=2,
                                           RT_tol=20,
                                           mz_Tol=1e-2,
                                           sample_id_col=-1,
                                           ms2_spec_id_col=0,
                                           ToAdd='mzML',
                                           min_Int_Frac=2,
                                           cos_tol=0.9)
            ms2_featuresDF=feat_ms2_Gauss(DataSet=DataSet,
                                          SummMS2=SummMS2,
                                          MS1IDVec=MS1IDVec,
                                          mz_std=2e-3,
                                          MS2_to_MS1_ratio=10,
                                          stdDistance=3,
                                          MaxCount=3,
                                          Points_for_regression=5,
                                          minSignals=7)
            DFname=ResultsFolder+'/'+datasetname+'.xlsx'
            ms2_featuresDF.to_excel(DFname)
            break
        except Exception as error:
            print("An exception occurred:", error)
            print('error',datasetname)
            break

21111_noSPE.mzML
An exception occurred: ms2_SpectralRedundancy() got an unexpected keyword argument 'SpectraList'
error 21111_noSPE.mzML
12313.mzML
An exception occurred: ms2_SpectralRedundancy() got an unexpected keyword argument 'SpectraList'
error 12313.mzML
12212.mzML
An exception occurred: ms2_SpectralRedundancy() got an unexpected keyword argument 'SpectraList'
error 12212.mzML
12311.mzML
An exception occurred: ms2_SpectralRedundancy() got an unexpected keyword argument 'SpectraList'
error 12311.mzML
13211.mzML
An exception occurred: ms2_SpectralRedundancy() got an unexpected keyword argument 'SpectraList'
error 13211.mzML
13112.mzML


KeyboardInterrupt: 

In [ ]:
ms2_featuresDF=feat_ms2_Gauss(DataSet=DataSet,SummMS2=SummMS2,MS1IDVec=MS1IDVec,mz_std=2e-3,MS2_to_MS1_ratio=10,stdDistance=3,MaxCount=3,Points_for_regression=5,minSignals=7)

NameError: name 'np' is not defined

In [3]:
import os
from pyopenms import *
def ChargeDataSet_in_AnotherFolder(DataSetName,DataFolder):
    DataSet=MSExperiment()
    MzMLFile().load(DataFolder+'/'+DataSetName, DataSet)
    return DataSet

In [10]:
Total

66

In [11]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ChargeDataSet import *
from Retrieve_csv_ms2Spectra import *
from ms2_SpectralRendundancy import *
from feat_ms2_Gauss import *
DataFolder='/data'
ResultsFolder=home+'/Results-20251123'
ms2Folder=home+'/ms2_spectra'
SFindicator='-ms2Summary.xlsx'
from All_ms2_spectra import *
from AllMS2Data import *
min_RT=0
max_RT=1500
min_mz=0
max_mz=1200
c=0
Total=len(os.listdir(DataFolder))
saveFolder='ms2_spectra'
for datasetname in os.listdir(DataFolder):
    while True:
        try:
            DataSet=ChargeDataSet_in_AnotherFolder(datasetname,DataFolder)
            SummMS2=AllMS2Data(DataSet=DataSet,min_RT=min_RT,max_RT=max_RT,min_mz=min_mz,max_mz=max_mz) 
            DataSetName=datasetname
            SpectraFoler=saveFolder+'/'+DataSetName.replace('.','') 
            SummMS2DF=pd.DataFrame(SummMS2,columns=['mz(Da)','RT(s)','id','maxInt','maxInt_frac'])
            SummMS2DF.to_excel(SpectraFoler+'-ms2Summary.xlsx')            
            SpectraList=All_ms2_spectra(DataSet=DataSet,SummMS2=SummMS2,DataSetName=datasetname,minInt=1e4,LogFileName='LogFile_ms2.csv',minPeaks=1,saveFolder='ms2_spectra',save=True)
            break
        except:
            print(datasetname)
            break
    print(c,int(c/Total*100))
    c+=1

0 0
1 1
2 3
3 4
4 6
5 7
6 9
7 10
8 12
9 13
10 15
11 16
12 18
13 19
14 21
15 22
16 24
17 25
18 27
19 28
20 30
21231_noSPE.mzML
21 31
22 33
23 34
24 36
25 37
26 39
27 40
28 42
29 43
30 45
31 46
32 48
33 50
34 51
35 53
36 54
37 56
38 57
39 59
40 60
41 62
42 63
43 65
44 66
45 68
46 69
47 71
48 72
49 74
50 75
51 77
52 78
53 80
54 81
55 83
56 84
57 86
58 87
59 89
60 90
61 92
62 93
63 95
64 96
65 98


In [6]:
DataFolder='/home/edwin/Documents/16.02/data'
BatchExtract_All_MS2_Spectra(DataFolder=DataFolder,
                             saveFolder='ms2_spectra-20260107',
                             min_RT=0,
                             max_RT=1500,
                             min_mz=0,
                             max_mz=1200,
                             minInt=1e4,
                             minPeaks=1,
                             LogFileName='LogFile_ms2-20260107.csv',
                             SFindicator='-ms2Summary.csv')

/home/edwin/0-GitHubProjects/Codding/ms2Gauss/Functions/Normal_Fit.py:10: OptimizeWarning: Covariance of the parameters could not be estimated
  GaussianParameters=list(curve_fit(GaussianPeak, xdata=PeakData[:,0], ydata=PeakData[:,1],p0=[mz,mz_std,I_total])[0])
/home/edwin/0-GitHubProjects/Codding/ms2Gauss/Functions/mz_Gauss_std.py:19: RuntimeWarning: invalid value encountered in sqrt
  mz_std=np.sqrt(-m/2)


0 0
1 1
2 3
3 4
4 6
5 7
6 9
7 10
8 12
9 13
10 15
11 16
12 18
13 19
14 21
15 22
16 24
17 25
18 27
19 28
20 30
FAILED: 21231_noSPE.mzML
21 31


/__w/OpenMS/OpenMS/OpenMS/src/openms/source/FORMAT/HANDLERS/XMLHandler.cpp(107): While loading '/home/edwin/Documents/16.02/data/21231_noSPE.mzML': input ended before all started tags were ended; last tag started is 'spectrumList'( in line 300987 column 1)


22 33
23 34
24 36
25 37
26 39
27 40
28 42
29 43
30 45
31 46
32 48
33 50
34 51
35 53
36 54
37 56
38 57
39 59
40 60
41 62
42 63
43 65
44 66
45 68
46 69
47 71
48 72
49 74
50 75
51 77
52 78
53 80
54 81
55 83
56 84
57 86
58 87
59 89
60 90
61 92
62 93
63 95
64 96
65 98


In [7]:
DataFolder='/home/edwin/Documents/16.02/data'
for datasetname in os.listdir(DataFolder):
    while True:
        try:
            DataSet=ChargeDataSet_in_AnotherFolder(datasetname,DataFolder)
            MS1IDVec=MS_L_IDs(DataSet=DataSet,Level=1)
            SummMS2=AllMS2Data(DataSet=DataSet)
            MS2_features=feat_ms2_Gauss(DataSet=DataSet,SummMS2=SummMS2,MS1IDVec=MS1IDVec)
            DF=pd.DataFrame(MS2_features)
            DFname=ResultsFolder+'/'+datasetname+'.xlsx'
            DF.to_excel(DFname)
            break
        except:
            print(datasetname)
            break

error
error
error
error


/home/edwin/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


error
error
error
error
error
error
error
error


/__w/OpenMS/OpenMS/OpenMS/src/openms/source/FORMAT/HANDLERS/XMLHandler.cpp(107): While loading '/home/edwin/Documents/16.02/data/21231_noSPE.mzML': input ended before all started tags were ended; last tag started is 'spectrumList'( in line 300987 column 1)


21231_noSPE.mzML
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
